# Deploying a Model using Azure Machine Learning

The steps to deploy any model are:

1. Register the model
2. Prepare an entry script
3. Prepare an inference configuration and a deployment configuration
4. Deploy the model locally to ensure everything works
5. Choose a compute target.
6. Re-deploy the model to the cloud
7. Test the resulting web service.


[You can learn more by reading these official docs](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where)

## 0. Connect to AML

In [110]:
import azureml.core
from azureml.core import Workspace, Dataset, Model, Environment
from azureml.core.model import InferenceConfig

subscription_id = '651f2ed5-6e2f-41d0-b533-0cd28801ef2a'
resource_group = 'data'
workspace_name = 'data-workspace05a7960'

ws = Workspace(subscription_id, resource_group, workspace_name)

print('Connected to workspace:', ws.name)

tags = {
    'source': 'tutorial',
    'production': False
}

Connected to workspace: data-workspace05a7960


## 1. Register a model

A typical situation for a deployed machine learning service is that you need the following components:

* resources representing the specific model that you want deployed (for example: a pytorch model file)
* code that you will be running in th service, that executes the model on a given input

Azure Machine Learning allows you to separate the deployment into two separate components, so that you can keep the same code, but merely update the model. We define the mechanism by which you upload a model separately from your code as "registering the model".

You can register a model by providing the local path of the model. You can provide the path of either a folder or a single file on your local machine.

In [132]:
# define a simple file as the specific 'model' we're going to use.
import pickle

model_parameters = {'name': 'tutorial-model',
        'parameters': {
            'weights': [0.6, 0.3, 0.1]
        }
    }
model_filename = 'tutorial_model.pkl'

with open(model_filename, 'wb') as handle:
    pickle.dump(model_parameters, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [97]:
offers = sorted([{'id': 2}, {'id': 3}, {"id": 1} ], key=lambda o: o['id'])
print(offers)

[{'id': 1}, {'id': 2}, {'id': 3}]


In [134]:
# register the model
model_properties = {
    'source': 'tutorial'
}
model = Model.register(ws, 
                       model_name=model_parameters['name'], 
                       model_path=model_filename, 
                       tags=tags
                      )

Registering model tutorial-model


In [79]:
print(model)

Model(workspace=Workspace.create(name='data-workspace05a7960', subscription_id='651f2ed5-6e2f-41d0-b533-0cd28801ef2a', resource_group='data'), name=tutorial-model, id=tutorial-model:4, version=4, tags={}, properties={'source': 'tutorial'})


## 2. Prepare an entry script

The entry script receives data submitted to a deployed web service and passes it to the model. It then returns the model's response to the client. The script is specific to your model. The entry script must understand the data that the model expects and returns.

> You can use the environment variable `AZUREML_MODEL_DIR` to locate your model that you registered earlier

In [46]:
%%writefile source_dir/entry.py 

# this is a very basic entry script
import json

def init():
    print('This is init')

def run(data):
    test = json.loads(data)
    print(f'received data {test}')
    return(f'test is {test}')

Overwriting source_dir/entry.py


In [99]:
%%writefile source_dir/entry.py 

# this is an actual entry script
import json
import pickle
import random
import os

def init():
    global parameters
    with open(os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'tutorial_model.pkl'), 'rb') as handle:
        parameters = pickle.load(handle)['parameters']

# request should have: offers: [],  features: {}
def run(request):
    print(request)
    recommendaton_request = json.loads(request)
    # Run inference
    recomendation = recommend(recommendaton_request)
    print(recomendation)
    return recomendation

def recommend(recommendaton_request):
    # sort the offers by ID
    offers = sorted(recommendaton_request['offers'], key=lambda o: o['id'])
    print(offers)
    features = recommendaton_request['features']
    return random.choices(offers, parameters['weights'] )


Overwriting source_dir/entry.py


## 3. Prepare an inference configuration and a deployment configuration

### Inference

An inference configuration describes the Docker container and files to use when initializing your web service. All of the files within your source directory, including subdirectories, will be zipped up and uploaded to the cloud when you deploy your web service.

### Deployment

A deployment configuration specifies the amount of memory and cores to reserve for your webservice will require in order to run, as well as configuration details of the underlying webservice. For example, a deployment configuration lets you specify that your service needs 2 gigabytes of memory, 2 CPU cores, 1 GPU core, and that you want to enable autoscaling.

In [81]:
env = Environment.from_pip_requirements(name='tutorial_environment', file_path="./model_requirements.txt")
inference_config = InferenceConfig(environment=env, source_directory='source_dir', entry_script='./entry.py')

In [113]:
# this creates a local webservice
from azureml.core.webservice import LocalWebservice
deployment_config = LocalWebservice.deploy_configuration(port=6789)

## 4. Deploy the model locally to ensure everything works

This part needs docker installed and running

In [100]:
service = Model.deploy(ws, "tutorial-service-local", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output=True)
print(service.get_logs())

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry 145d1c48df42405ba082e5e8f747442e.azurecr.io
Logging into Docker registry 145d1c48df42405ba082e5e8f747442e.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM 145d1c48df42405ba082e5e8f747442e.azurecr.io/azureml/azureml_984ae42c92ab7e15d7808a45a9ac459f
 ---> 3b27530f31be
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 2a82f9441e8c
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjY1MWYyZWQ1LTZlMmYtNDFkMC1iNTMzLTBjZDI4ODAxZWYyYSIsInJlc291cmNlR3JvdXBOYW1lIjoiZGF0YSIsImFjY291bnROYW1lIjoiZGF0YS13b3Jrc3BhY2UwNWE3OTYwIiwid29ya3NwYWNlSWQiOiIxNDVkMWM0OC1kZjQyLTQwNWItYTA4Mi1lNWU4Zjc0NzQ0MmUifSwibW9kZWxzIjp7fSwibW9kZWxzSW5mbyI6e319 | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 92812a98acb5
 ---> fc24f72280dd
Step 4/5 : RUN mv '/var/azureml-app/tmptagh3v37.py' /var/azureml-app/main.py
 ---> Running in 19633732fa1

Call the local docker container to check that it works

In [103]:
# check the model works
import requests
import json

uri = service.scoring_uri
requests.get('http://localhost:6789')
headers = {'Content-Type': 'application/json'}
data = {"features": {
            "country": "Australia"
        }, "offers": [
            {
                "id": 1,
                "price": 10,
                "name": "Bahn Mi"
            },
            {
                "id": 2,
                "price": 20,
                "name": "Fish Stew"
            },
            {
                "id": 3,
                "price": 30,
                "name": "Steak Tartar"
            }
        ]}
data = json.dumps(data)
response = requests.post(uri, data=data, headers=headers)
print('model responded with:')
print(response.json())

model responded with:
[{'id': 1, 'price': 10, 'name': 'Bahn Mi'}]


## 5. Choose a compute target.

[Learn more about choosing a target](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where?tabs=python#choose-a-compute-target)

Options are:
* Local web service: for testing and debugging
* Azure Kubernetes Services: High scale production (probably don't do this without bigger discussion)
* Azure Container Instances: Low scale, less than 48GB RAM
* Azure Machine Learning compute cluster: best for batch inferencing


## 6. Re-deploy the model to the cloud

Deploy to an Azure Container Instanec

In [116]:
from azureml.core.webservice import AciWebservice
aci_deployment_config = AciWebservice.deploy_configuration(cpu_cores = 0.5, 
                                                           memory_gb = 1, 
                                                           tags=tags, 
                                                           auth_enabled=True)

In [118]:
# this cell creates a new ACI deployed into Azure.
service = Model.deploy(ws, 
                       "tutorial-service-aci", 
                       [model], 
                       inference_config, 
                       aci_deployment_config)

service.wait_for_deployment(show_output=True)
print(service.get_logs())

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-05-06 11:27:44+10:00 Creating Container Registry if not exists.
2021-05-06 11:27:44+10:00 Registering the environment.
2021-05-06 11:27:45+10:00 Use the existing image.
2021-05-06 11:27:45+10:00 Generating deployment configuration.
2021-05-06 11:27:47+10:00 Submitting deployment to compute..
2021-05-06 11:27:54+10:00 Checking the status of deployment tutorial-service-aci..
2021-05-06 11:30:08+10:00 Checking the status of inference endpoint tutorial-service-aci.
Succeeded
ACI service creation operation finished, operation "Succeeded"
2021-05-06T01:29:59,991399800+00:00 - rsyslog/run 
2021-05-06T01:29:59,982298400+00:00 - iot-server/run 
2021-05-06T01:29:59,994369000+00:00 - gunicorn/run 
2021-05-06T01:30:00,158317300+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_abe832532d68e7634ad48db

## 7. Test the resulting web service.

When you deploy remotely, you may have key authentication enabled. The example below shows how to get your service key with Python in order to make an inference request.

In [121]:
import requests
import json
from azureml.core import Webservice

service = Webservice(workspace=ws, name='tutorial-service-aci')
scoring_uri = service.scoring_uri

# If the service is authenticated, set the key or token
primary_key, _ = service.get_keys()

# Set the appropriate headers
headers = {'Content-Type': 'application/json'}
headers['Authorization'] = f'Bearer {primary_key}'

# Make the request and display the response and logs
data = {"features": {
            "country": "Australia"
        }, "offers": [
            {
                "id": 1,
                "price": 10,
                "name": "Bahn Mi"
            },
            {
                "id": 2,
                "price": 20,
                "name": "Fish Stew"
            },
            {
                "id": 3,
                "price": 30,
                "name": "Steak Tartar"
            }
        ]}

data = json.dumps(data)
resp = requests.post(scoring_uri, data=data, headers=headers)
print('the model responded:')
print(resp.text)


the model responded:
[{"id": 1, "price": 10, "name": "Bahn Mi"}]


You can get the logs from the remote ACI

In [122]:
print(service.get_logs())

2021-05-06T01:29:59,991399800+00:00 - rsyslog/run 
2021-05-06T01:29:59,982298400+00:00 - iot-server/run 
2021-05-06T01:29:59,994369000+00:00 - gunicorn/run 
2021-05-06T01:30:00,158317300+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_abe832532d68e7634ad48db5ac241baf/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_abe832532d68e7634ad48db5ac241baf/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_abe832532d68e7634ad48db5ac241baf/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_abe832532d68e7634ad48db5ac241baf/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_abe832532d68e7634ad48db5ac241baf/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
EdgeHubC

## WELL DONE!

You deployed your first model!

The next part is making it intelligent.

But first, clean up the resources you created.

In [125]:
# delete the ACI service you created
service.delete()

No service with name tutorial-service-aci found to delete.


In [136]:
# delete all the models we created
models = Model.list(ws, name=model_parameters['name'])
for m in models:
    print('deleting', model.name, 'version:' , model.version)
    m.delete()